In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import pandas as pd

import numpy as np 

import create_model
import pickle
import time 
import copy
import random
import data_formatting

In [2]:
#dataset = 'enron'

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl')

dataset = 'twitter'

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl')

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl')

vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_19_sample_21946_lem.pkl', 'rb'))
df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_19_sample_21946_lem.pkl')

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_25_sample_1901567_full.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_25_sample_1901567_full.pkl')

In [3]:
inv_map = data_formatting.createInvMap(vocab_dict)

In [4]:
df_all['alpha_Pair_0_encoding'] = df_all['alpha_Pair_0_tokens'].apply(lambda x: data_formatting.encodeSent(x, vocab_dict))
df_all['alpha_Pair_1_encoding'] = df_all['alpha_Pair_1_tokens'].apply(lambda x: data_formatting.encodeSent(x, vocab_dict))
df_all['Index'] = df_all.index.values

In [5]:
df_all_train = df_all.sample(frac=0.97, random_state=1)

df_all_dev = df_all[df_all['Index'].isin(df_all_train['Index'].values) == False]

df_all_test = df_all_dev.sample(frac=0.10, random_state=1)

df_all_dev = df_all_dev[df_all_dev['Index'].isin(df_all_test['Index'].values) == False]

In [6]:
print (df_all.shape[0], df_all_train.shape[0],  df_all_dev.shape[0], df_all_test.shape[0], len(vocab_dict))

17690 17159 478 53 17322


In [7]:
train_data = data_formatting.prepare_train_batch(df_all_train['alpha_Pair_0_encoding'].values, 
                                                    df_all_train['alpha_Pair_1_encoding'].values)

dev_data = data_formatting.prepare_train_batch(df_all_dev['alpha_Pair_0_encoding'].values, 
                                                    df_all_dev['alpha_Pair_1_encoding'].values)

test_data = data_formatting.prepare_train_batch(df_all_test['alpha_Pair_0_encoding'].values, 
                                                    df_all_test['alpha_Pair_1_encoding'].values)

In [8]:
train_model_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1, 'minibatch_size':64, 'n_threads':128,
          'beam_width':10, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

In [9]:
dev_model_params = train_model_params
dev_model_params['encoder_input_keep'] = 1
dev_model_params['encoder_output_keep'] = 1
dev_model_params['decoder_input_keep'] = 1
dev_model_params['decoder_output_keep'] = 1

In [10]:
training_params = { 'vocab_lower':3, 'vocab_upper':train_model_params['vocab_size']-1, 
                    'n_epochs':500000}

In [11]:
#enc input           > ['i', 'll', 'be', 'there', 'tomorrow', '<EOS>']
#dec input           > ['last', 'game', 'of', 'the', 'regular', 'season', '<EOS>']

In [12]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import nn_ops
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.util import nest
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import tensor_array_ops

In [13]:
tf.reset_default_graph()

inf_model_1_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1, 'minibatch_size':64, 'n_threads':128,
          'beam_width':1, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

#inf_model_2_params = {'n_cells':2, 'num_layers':1, 'embedding_size':5, 
#            'vocab_size':7,
#          'minibatch_size':64, 'n_threads':128,
#          'beam_width':3, 'limit_decode_steps': None,
#          'encoder_input_keep':1, 'decoder_input_keep':1,
#          'encoder_output_keep':1, 'decoder_output_keep':1,
#         }


inf_model_2_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1,
          'minibatch_size':64, 'n_threads':128,
          'beam_width':3, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

#with tf.variable_scope('training_model'):

#    inf_model_1 = create_model.Model(inf_model_1_params, 'infer', None)
    
#with tf.variable_scope('training_model'):

#    inf_model_2 = create_model.Model(inf_model_2_params, 'infer', None)   

with tf.variable_scope('training_model'):

    inf_train = create_model.Model(inf_model_2_params, 'debug', None)   

ValueError: slice index 2 of dimension 0 out of bounds. for 'training_model/while/strided_slice_7' (op: 'StridedSlice') with input shapes: [2], [1], [1], [1] and with computed input tensors: input[1] = <2>, input[2] = <3>, input[3] = <1>.

In [ ]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    #saver = tf.train.Saver()
   
    #saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run(inf_train.decode_loop, 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

In [19]:
q_0[4]

array([[-0.00019024, -0.10373823, -0.00839701, ...,  0.00793328,
        -0.0569768 ,  0.05554437],
       [-0.00019024, -0.10373823, -0.00839701, ...,  0.00793328,
        -0.0569768 ,  0.05554437],
       [-0.00019024, -0.10373822, -0.00839701, ...,  0.00793327,
        -0.0569768 ,  0.05554438]], dtype=float32)

In [ ]:
# Some tensor we want to print the value of
x = tf.placeholder(tf.float32, shape=[2, 2, 2])
a = np.array([[[1.,1.], [1.,1.]], [[2.,2.], [2.,2.]]])

m = tf.Print(x,[x])

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    m_eval = m.eval(session=sess,feed_dict={x: a})

In [17]:
batch_size = inf_model_2.inference_decoder._batch_size
beam_width = inf_model_2.inference_decoder._beam_width
end_token = inf_model_2.inference_decoder._end_token
length_penalty_weight = inf_model_2.inference_decoder._length_penalty_weight

In [18]:
name= None

In [19]:
time = 0

In [20]:
finished, first_inputs, initial_state = inf_model_2.inference_decoder.initialize()

In [21]:
first_inputs, initial_state

(<tf.Tensor 'training_model/embedding_lookup_1:0' shape=(?, 3, 512) dtype=float32>,
 BeamSearchDecoderState(cell_state=(LSTMStateTuple(c=<tf.Tensor 'training_model/Reshape:0' shape=(?, 3, 256) dtype=float32>, h=<tf.Tensor 'training_model/Reshape_1:0' shape=(?, 3, 256) dtype=float32>), AttentionWrapperState(cell_state=LSTMStateTuple(c=<tf.Tensor 'training_model/Reshape_2:0' shape=(?, 3, 256) dtype=float32>, h=<tf.Tensor 'training_model/Reshape_3:0' shape=(?, 3, 256) dtype=float32>), attention=<tf.Tensor 'training_model/Reshape_4:0' shape=(?, 3, 256) dtype=float32>, time=<tf.Tensor 'training_model/AttentionWrapperZeroState/zeros:0' shape=() dtype=int32>, alignments=<tf.Tensor 'training_model/Reshape_5:0' shape=(?, 3, ?) dtype=float32>, alignment_history=())), log_probs=<tf.Tensor 'zeros:0' shape=(?, 3) dtype=float32>, finished=<tf.Tensor 'training_model/zeros:0' shape=(?, 3) dtype=bool>, lengths=<tf.Tensor 'zeros_1:0' shape=(?, 3) dtype=int32>))

In [22]:
#first_inputs
#These are the word embeddings of the initial token (which for initialize should be from the encoder?)
#Shape is [batch_size, embedding_size]
#initial_state
#This is the initial state of the decoder, a tuple consisting of 
#1. the LSTM cell and hiddens states
#2. The attention states

In [23]:
#When we pass time !=0 to inference decoder, even with initial step results we get non-zero parent ids

#beam_search_output, beam_search_state, next_inputs, finished = \
#                                                inf_model_2.inference_decoder.step(time, first_inputs, initial_state)

In [24]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([first_inputs, initial_state, batch_size], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user, input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user), len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [25]:
q_0[0].shape

(2, 3, 512)

In [26]:
#with ops.name_scope(name, "BeamSearchDecoderStep", (time, first_inputs, initial_state)):
    
cell_state = initial_state.cell_state

first_inputs = nest.map_structure(lambda inp: inf_model_2.inference_decoder._merge_batch_beams(inp, s=inp.shape[2:]), 
                                  first_inputs)

cell_state = nest.map_structure(inf_model_2.inference_decoder._maybe_merge_batch_beams, cell_state, 
                                inf_model_2.inference_decoder._cell.state_size)

In [27]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([first_inputs, initial_state], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user, input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user), len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [28]:
q_0[0].shape

(6, 512)

In [29]:
##This goes through _cell first
#The output of cell_outputs is also present in the next_cell_state, the cell_outputs is the attention

#The cell_state includes, the cell and hidden states of the LSTM, the attention states, the alignment history, and timestep, 
#what does alignment history and attention states really mean? need to internalize...

cell_outputs, next_cell_state = inf_model_2.inference_decoder._cell(first_inputs, cell_state)

In [30]:
#These functions convert the shape of the cell outputs and states from [batch_size, beam_width, embedding_size]
#to [batch_size*beam_width, embedding_size], essentially, reducing a dimensionality by flattening the nested structure
#the map structure fn is used because cell state is a complex tuple structure, 
#outputs is a simpe structure (should be able to just call flatten?)
cell_outputs = nest.map_structure(lambda out: inf_model_2.inference_decoder._split_batch_beams(out, out.shape[1:]), 
                                  cell_outputs)

next_cell_state = nest.map_structure(inf_model_2.inference_decoder._maybe_split_batch_beams, next_cell_state, 
                                     inf_model_2.inference_decoder._cell.state_size)

if inf_model_2.inference_decoder._output_layer is not None:
    cell_outputs = inf_model_2.inference_decoder._output_layer(cell_outputs)

In [31]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([cell_outputs, next_cell_state], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [32]:
q_0[0][0]

array([[-15.48709488,  -8.94940948,  -9.60930347, ...,   1.44873977,
         -7.69063234, -10.08348083],
       [-15.48709869,  -8.94941139,  -9.60930443, ...,   1.44873857,
         -7.69063282, -10.08348083],
       [-15.48709869,  -8.94941139,  -9.60930347, ...,   1.44873834,
         -7.69063234, -10.08348179]], dtype=float32)

In [33]:
#cell_outputs_rnn_call, next_cell_state_rnn_call = inf_model_2.inference_decoder._cell.call(first_inputs, cell_state)

#cell_outputs_rnn_call_call, next_cell_state_rnn_call_call = inf_model_2.inference_decoder._cell.call(cell_outputs_rnn_call, 
#                                                                                                     next_cell_state_rnn_call)

In [34]:
sess.close()

In [35]:
static_batch_size = tensor_util.constant_value(batch_size)

prediction_lengths = initial_state.lengths
previously_finished = initial_state.finished

In [36]:
end_token = data_formatting.EOS

In [37]:
def _check_maybe(t):
  if isinstance(t, tensor_array_ops.TensorArray):
    raise TypeError(
        "TensorArray state is not supported by BeamSearchDecoder: %s" % t.name)
  if t.shape.ndims is None:
    raise ValueError(
        "Expected tensor (%s) to have known rank, but ndims == None." % t)

def _maybe_tensor_gather_helper(gather_indices, gather_from, batch_size,
                                range_size, gather_shape):
  """Maybe applies _tensor_gather_helper.
  This applies _tensor_gather_helper when the gather_from dims is at least as
  big as the length of gather_shape. This is used in conjunction with nest so
  that we don't apply _tensor_gather_helper to inapplicable values like scalars.
  Args:
    gather_indices: The tensor indices that we use to gather.
    gather_from: The tensor that we are gathering from.
    batch_size: The batch size.
    range_size: The number of values in each range. Likely equal to beam_width.
    gather_shape: What we should reshape gather_from to in order to preserve the
      correct values. An example is when gather_from is the attention from an
      AttentionWrapperState with shape [batch_size, beam_width, attention_size].
      There, we want to preserve the attention_size elements, so gather_shape is
      [batch_size * beam_width, -1]. Then, upon reshape, we still have the
      attention_size as desired.
  Returns:
    output: Gathered tensor of shape tf.shape(gather_from)[:1+len(gather_shape)]
      or the original tensor if its dimensions are too small.
  """
  _check_maybe(gather_from)
  if gather_from.shape.ndims >= len(gather_shape):
    return _tensor_gather_helper(
        gather_indices=gather_indices,
        gather_from=gather_from,
        batch_size=batch_size,
        range_size=range_size,
        gather_shape=gather_shape)
  else:
    return gather_from

def _tensor_gather_helper(gather_indices, gather_from, batch_size,
                          range_size, gather_shape):
  """Helper for gathering the right indices from the tensor.
  This works by reshaping gather_from to gather_shape (e.g. [-1]) and then
  gathering from that according to the gather_indices, which are offset by
  the right amounts in order to preserve the batch order.
  Args:
    gather_indices: The tensor indices that we use to gather.
    gather_from: The tensor that we are gathering from.
    batch_size: The input batch size.
    range_size: The number of values in each range. Likely equal to beam_width.
    gather_shape: What we should reshape gather_from to in order to preserve the
      correct values. An example is when gather_from is the attention from an
      AttentionWrapperState with shape [batch_size, beam_width, attention_size].
      There, we want to preserve the attention_size elements, so gather_shape is
      [batch_size * beam_width, -1]. Then, upon reshape, we still have the
      attention_size as desired.
  Returns:
    output: Gathered tensor of shape tf.shape(gather_from)[:1+len(gather_shape)]
  """
  range_ = array_ops.expand_dims(math_ops.range(batch_size) * range_size, 1)
  gather_indices = array_ops.reshape(gather_indices + range_, [-1])
  output = array_ops.gather(
      array_ops.reshape(gather_from, gather_shape), gather_indices)
  final_shape = array_ops.shape(gather_from)[:1 + len(gather_shape)]
  static_batch_size = tensor_util.constant_value(batch_size)
  final_static_shape = (tensor_shape.TensorShape([static_batch_size])
                        .concatenate(
                            gather_from.shape[1:1 + len(gather_shape)]))
  output = array_ops.reshape(output, final_shape)
  output.set_shape(final_static_shape)
  return output
#Have to understand how this works!
def _mask_probs(probs, eos_token, finished):
  """Masks log probabilities.
  The result is that finished beams allocate all probability mass to eos and
  unfinished beams remain unchanged.
  Args:
    probs: Log probabiltiies of shape `[batch_size, beam_width, vocab_size]`
    eos_token: An int32 id corresponding to the EOS token to allocate
      probability to.
    finished: A boolean tensor of shape `[batch_size, beam_width]` that
      specifies which
      elements in the beam are finished already.
  Returns:
    A tensor of shape `[batch_size, beam_width, vocab_size]`, where unfinished
    beams stay unchanged and finished beams are replaced with a tensor with all
    probability on the EOS token.
  """
  vocab_size = array_ops.shape(probs)[2]
  finished_mask = array_ops.expand_dims(
      math_ops.to_float(1. - math_ops.to_float(finished)), 2)
  # These examples are not finished and we leave them
  non_finished_examples = finished_mask * probs
  # All finished examples are replaced with a vector that has all
  # probability on EOS
  finished_row = array_ops.one_hot(
      eos_token,
      vocab_size,
      dtype=probs.dtype,
      on_value=0.,
      off_value=probs.dtype.min)
  finished_examples = (1. - finished_mask) * finished_row
  return finished_examples + non_finished_examples

In [38]:
logits = cell_outputs
step_log_probs = nn_ops.log_softmax(logits)

In [39]:
step_log_probs

<tf.Tensor 'Reshape_15:0' shape=(?, 3, 17323) dtype=float32>

In [40]:
tf.shape(step_log_probs)[0]

<tf.Tensor 'strided_slice_14:0' shape=() dtype=int32>

In [222]:

step_log_probs = _mask_probs(step_log_probs, end_token, previously_finished)
total_probs = array_ops.expand_dims(initial_state.log_probs, 2) + step_log_probs

In [223]:
vocab_size = logits.shape[-1].value or array_ops.shape(logits)[-1]

lengths_to_add = array_ops.one_hot(
      indices=array_ops.tile(
          array_ops.reshape(end_token, [1, 1]), [batch_size, beam_width]),
      depth=vocab_size,
      on_value=0,
      off_value=1)

In [271]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([total_probs, array_ops.expand_dims(initial_state.log_probs, 2), step_log_probs], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [279]:
q_0[2][0][2] + q_0[1][0][2]

array([-30.88870049, -10.23493576, -46.66865921, ..., -40.9866333 ,
       -40.04515076, -29.67861557], dtype=float32)

In [275]:
q_0[0][0]

array([[-39.28241348, -10.5441494 , -41.33394623, ..., -33.4271698 ,
        -35.07487869, -32.33093262],
       [-19.30293655,  -7.79374218, -34.72539139, ..., -33.93330765,
        -27.5273056 , -22.93684769],
       [-30.88870049, -10.23493576, -46.66865921, ..., -40.9866333 ,
        -40.04515076, -29.67861557]], dtype=float32)

In [227]:
sess.close()

In [228]:
add_mask = (1 - math_ops.to_int32(previously_finished))

lengths_to_add = array_ops.expand_dims(add_mask, 2) * lengths_to_add

new_prediction_lengths = (lengths_to_add + array_ops.expand_dims(prediction_lengths, 2))

In [229]:
scores = total_probs

In [230]:
scores_shape = array_ops.shape(scores)

In [231]:
time = ops.convert_to_tensor(time, name="time")

In [232]:
scores_flat = control_flow_ops.cond(time > 0,
      lambda: array_ops.reshape(scores, [batch_size, -1]), lambda: scores[:, 0])

In [233]:
num_available_beam = control_flow_ops.cond(
      time > 0, lambda: math_ops.reduce_prod(scores_shape[1:]),
      lambda: math_ops.reduce_prod(scores_shape[2:]))

In [234]:
# Pick the next beams according to the specified successors function
next_beam_size = math_ops.minimum(
  ops.convert_to_tensor(beam_width, dtype=dtypes.int32, name="beam_width"),
  num_available_beam)

In [235]:
next_beam_scores, word_indices = nn_ops.top_k(scores_flat, k=next_beam_size)
next_beam_scores.set_shape([static_batch_size, beam_width])
word_indices.set_shape([static_batch_size, beam_width])

In [236]:
next_beam_probs = _tensor_gather_helper(
      gather_indices=word_indices,
      gather_from=scores,
      batch_size=batch_size,
      range_size=beam_width * vocab_size,
      gather_shape=[-1])

In [237]:
#Here, word indices represents the positions of the flattened list of beams, which goes from 1 to beam_width*vocab_size
#thus the word indices obtained are not the word indices of the 1 to N vocab but the flattened out list of logprobs from 1 to 
#beam_width*vocab_size
next_word_ids = math_ops.to_int32(word_indices % vocab_size)
#The beam_ids represent which beams these word indices (and thereby log prob values belong to), as
#we have word indices extracted out of 1 to beam_width*vocab_size
next_beam_ids = math_ops.to_int32(word_indices / vocab_size)

In [251]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([next_word_ids, next_beam_ids, scores, scores_flat, next_beam_scores, next_beam_probs,
                    word_indices, next_beam_size], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [249]:
q_0[2][0]

array([[-39.2824173 , -10.54414368, -41.33394623, ..., -33.4271698 ,
        -35.07487488, -32.33093262],
       [-19.30294037,  -7.7937398 , -34.72540283, ..., -33.93331528,
        -27.52731133, -22.93685341],
       [-30.88870049, -10.23493862, -46.66866684, ..., -40.98663712,
        -40.04515076, -29.67861748]], dtype=float32)

In [257]:
q_0[3][0][np.argsort(q_0[3][0])[-3:]]

array([-2.55750656, -1.63940859, -1.51259661], dtype=float32)

In [248]:
q_0[3][0][:vocab_size], q_0[3][0][vocab_size:vocab_size*2], q_0[3][0][vocab_size*2:]

(array([-39.2824173 , -10.54414368, -41.33394623, ..., -33.4271698 ,
        -35.07487488, -32.33093262], dtype=float32),
 array([-19.30294037,  -7.7937398 , -34.72540283, ..., -33.93331528,
        -27.52731133, -22.93685341], dtype=float32),
 array([-30.88870049, -10.23493862, -46.66866684, ..., -40.98663712,
        -40.04515076, -29.67861748], dtype=float32))

In [252]:
q_0[4], q_0[5]

(array([[-1.51259661, -1.63940859, -2.55750656]], dtype=float32),
 array([[-1.51259661, -1.63940859, -2.55750656]], dtype=float32),
 array([[-1.51259661, -1.63940859, -2.55750656]], dtype=float32))

In [250]:
q_0[0], q_0[1]

(array([[15014,  3722, 16662]]), array([[0, 0, 1]]))

In [134]:
previously_finished = _tensor_gather_helper(
      gather_indices=next_beam_ids,
      gather_from=previously_finished,
      batch_size=batch_size,
      range_size=beam_width,
      gather_shape=[-1])

next_finished = math_ops.logical_or(previously_finished,
                                      math_ops.equal(next_word_ids, end_token))

In [135]:
next_finished = math_ops.logical_or(previously_finished, math_ops.equal(next_word_ids, end_token))

In [136]:
lengths_to_add = math_ops.to_int32(math_ops.not_equal(next_word_ids, end_token))
lengths_to_add = (1 - math_ops.to_int32(next_finished)) * lengths_to_add

In [137]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([previously_finished, next_finished, lengths_to_add], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [138]:
q_0

[array([[False, False, False]], dtype=bool),
 array([[False, False, False]], dtype=bool),
 array([[1, 1, 1]])]

In [139]:
next_prediction_len = _tensor_gather_helper(
      gather_indices=next_beam_ids,
      gather_from=initial_state.lengths,
      batch_size=batch_size,
      range_size=beam_width,
      gather_shape=[-1])

In [140]:
next_prediction_len += lengths_to_add

In [141]:
with tf.Session() as sess:
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_prediction_len],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})    

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [142]:
q_1

[array([[1, 1, 1]])]

In [143]:
next_cell_state = nest.map_structure(
      lambda gather_from: _maybe_tensor_gather_helper(
          gather_indices=next_beam_ids,
          gather_from=gather_from,
          batch_size=batch_size,
          range_size=beam_width,
          gather_shape=[batch_size * beam_width, -1]),
      next_cell_state)

In [144]:
with tf.Session() as sess:
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_cell_state],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})    

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [145]:
#These are the returns, next_state and output to the step function, but how exactly do we transpose this back and push it
#into cell? Moreover what is teh mechanics that determines when the beams become different? I.E. are no longer identical

In [146]:
next_state = tf.contrib.seq2seq.BeamSearchDecoderState(
      cell_state=next_cell_state,
      log_probs=next_beam_probs,
      lengths=next_prediction_len,
      finished=next_finished)

In [147]:
output = tf.contrib.seq2seq.BeamSearchDecoderOutput(
      scores=next_beam_scores,
      predicted_ids=next_word_ids,
      parent_ids=next_beam_ids)

In [148]:
with tf.Session() as sess:
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_state, output, next_beam_scores],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})    

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [149]:
beam_search_state = next_state
beam_search_output = output

In [150]:
finished = beam_search_state.finished

In [151]:
sample_ids = beam_search_output.predicted_ids

In [152]:
next_inputs = control_flow_ops.cond(math_ops.reduce_all(finished), 
                                    lambda: inf_model_2.inference_decoder._start_inputs,
                                    lambda: inf_model_2.inference_decoder._embedding_fn(sample_ids))

In [155]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    #sess.run(tf.global_variables_initializer())

    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_inputs, beam_search_state],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]}) 

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [202]:
q_0[0].shape, q_1[0][0].shape

((3, 512), (3, 512))

In [176]:
q_0[1][3].shape, q_1[1][3].shape

((1, 3), (1, 3))

In [177]:
q_0[1][2].shape, q_1[1][2].shape

((1, 3), (1, 3))

In [178]:
q_0[1][1].shape, q_1[1][1].shape

((1, 3), (1, 3))

In [181]:
len(q_0[1][0]), len(q_1[1][0])

(2, 2)

In [209]:
#next_inputs
#These are the chosen word ids which we have performed the embedding look up for
first_inputs = next_inputs[0]

#next_states
#these are the next states of the decoder network which pass back through the process
initial_state = next_state

In [204]:
finished = finished

In [217]:
time =1